In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Scaling Criteo: Training with Merlin Models

This notebook is created using the latest stable [merlin-tensorflow-training](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow-training/tags) container.


## Overview

The [Criteo 1TB Click Logs dataset](https://ailab.criteo.com/download-criteo-1tb-click-logs-dataset/) is a popular dataset in the recommender system community as it is one of the largest, public available dataset. It contains ~1.3 TB of uncompressed click logs containing over four billion samples spanning 24 days.

We will train Facebook's [deep learning recommendation model (DLRM)](https://arxiv.org/abs/1906.00091) architecture with Merlin Models. We will assume you are familiar with Merlin Models' API and features. Otherwise, we recommend to start with the [Merlin Models examples](https://github.com/NVIDIA-Merlin/models/tree/main/examples).

## Learning objectives

- Train a DLRM architecture with Merlin Models on a large dataset

## Training a DLRM model

Let's start with importing the libraries that we'll use in this notebook.

In [2]:
import os

import glob
import time
import merlin.models.tf as mm
from merlin.io.dataset import Dataset

from merlin.schema import Tags
import tensorflow as tf

2022-06-22 13:28:57.068091: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-22 13:28:59.393436: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16255 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:06:00.0, compute capability: 7.0


Define the path to directories which contains the processed data.

In [3]:
input_path = os.environ.get("INPUT_DATA_DIR", "/tmp/criteo/")

# path to processed data
PATH_TO_TRAIN_DATA = sorted(glob.glob(os.path.join(input_path, "train", "*.parquet")))
PATH_TO_VALID_DATA = sorted(glob.glob(os.path.join(input_path, "valid", "*.parquet")))

PATH_TO_TRAIN_DATA, PATH_TO_VALID_DATA

(['/tmp/criteo/train/part_0.parquet'], ['/tmp/criteo/valid/part_0.parquet'])

We define some hyperparameters for the model architecture.

In [4]:
BATCH_SIZE = int(os.environ.get("BATCH_SIZE", 64 * 1024))
EMBEDDING_SIZE = 32
EPOCHS = 1
LR = 0.01
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=LR)

We will use Merlin Dataset object to initalize the dataloaders. It provides a dataset schema to initialize the model architectures. The [Merlin Models examples](https://github.com/NVIDIA-Merlin/models/tree/main/examples) will explain more details.

In [5]:
train = Dataset(PATH_TO_TRAIN_DATA, part_mem_fraction=0.08)
valid = Dataset(PATH_TO_VALID_DATA, part_mem_fraction=0.08)

/usr/lib/python3.8/site-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


We initalize the DLRM architecture with Merlin Models.

In [6]:
model = mm.DLRMModel(
    train.schema,
    embedding_dim=EMBEDDING_SIZE,
    bottom_block=mm.MLPBlock([128, EMBEDDING_SIZE]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    )
)

We compile and train our model.

In [7]:
%%time

model.compile(optimizer=OPTIMIZER, run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=BATCH_SIZE, epochs=EPOCHS)

2989/2989 [==============================] - 375s 114ms/step - loss: 0.1580 - precision: 0.0383 - recall: 0.0016 - binary_accuracy: 0.9667 - auc: 0.5820 - regularization_loss: 0.0000e+00 - val_loss: 0.1438 - val_precision: 0.0905 - val_recall: 7.4995e-05 - val_binary_accuracy: 0.9661 - val_auc: 0.6481 - val_regularization_loss: 0.0000e+00
CPU times: user 7min 16s, sys: 6min 46s, total: 14min 3s
Wall time: 6min 35s


## Evaluate the model

Finally, we can evaluate our model on the validation dataset.

In [8]:
eval_metrics = model.evaluate(valid, batch_size=BATCH_SIZE, return_dict=True)
eval_metrics

2633/2633 [==============================] - 108s 38ms/step - loss: 0.1438 - precision: 0.0905 - recall: 7.4995e-05 - binary_accuracy: 0.9661 - auc: 0.6481 - regularization_loss: 0.0000e+00


{'loss': 0.14381085336208344,
 'precision': 0.09051457047462463,
 'recall': 7.49945902498439e-05,
 'binary_accuracy': 0.9661294221878052,
 'auc': 0.6481243968009949,
 'regularization_loss': 0.0}

## Save the model

We save the model to disk.

In [9]:
model.save(os.path.join(input_path, "dlrm"))

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

...

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

INFO:tensorflow:Unsupported signature for serialization: (({'C1': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C1'), 'C2': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C2'), 'C3': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C3'), 'C4': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C4'), 'C5': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C5'), 'C6': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C6'), 'C7': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C7'), 'C8': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C8'), 'C9': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C9'), 'C10': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C10'), 'C11': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C11'), 'C12': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C12'), 'C13': TensorSpec(shape=(None, 1), dtype=tf.int32, name='inputs/C13'), 'C14': TensorSpec(shape=(None, 1), 

INFO:tensorflow:Assets written to: /tmp/criteo/dlrm/assets


INFO:tensorflow:Assets written to: /tmp/criteo/dlrm/assets
